In [3]:
import pandas as pd

df = pd.read_csv("data/polling_data.csv")

In [4]:
# Take a peak at the data landscape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18095 entries, 0 to 18094
Data columns (total 52 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   poll_id                    18095 non-null  int64  
 1   pollster_id                18095 non-null  int64  
 2   pollster                   18095 non-null  object 
 3   sponsor_ids                8482 non-null   object 
 4   sponsors                   8482 non-null   object 
 5   display_name               18095 non-null  object 
 6   pollster_rating_id         18095 non-null  int64  
 7   pollster_rating_name       18095 non-null  object 
 8   numeric_grade              15901 non-null  float64
 9   pollscore                  15915 non-null  float64
 10  methodology                16995 non-null  object 
 11  transparency_score         14886 non-null  float64
 12  state                      10071 non-null  object 
 13  start_date                 18095 non-null  obj

In [5]:
# Filters
battleground_states = ["Georgia", "North Carolina", "Pennsylvania", "Michigan", "Wisconsin", "Nevada", "Arizona"]
mask = (
        (df["answer"].isin(["Trump", "Harris"])) &
        (df["state"].isin(battleground_states))
)

# Copy original data for cleaning
clean_data = df[mask].copy()
clean_data.to_csv("data/raw_battleground_data.csv", index=False)
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3331 entries, 7 to 18060
Data columns (total 52 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   poll_id                    3331 non-null   int64  
 1   pollster_id                3331 non-null   int64  
 2   pollster                   3331 non-null   object 
 3   sponsor_ids                1820 non-null   object 
 4   sponsors                   1820 non-null   object 
 5   display_name               3331 non-null   object 
 6   pollster_rating_id         3331 non-null   int64  
 7   pollster_rating_name       3331 non-null   object 
 8   numeric_grade              2747 non-null   float64
 9   pollscore                  2761 non-null   float64
 10  methodology                2992 non-null   object 
 11  transparency_score         3246 non-null   float64
 12  state                      3331 non-null   object 
 13  start_date                 3331 non-null   object 
 

In [6]:
# Get polling averages for each battleground state
polling_averages = clean_data.groupby(["state", "answer"])["pct"].mean().reset_index()
print(polling_averages)

             state  answer        pct
0          Arizona  Harris  46.646402
1          Arizona   Trump  46.792305
2          Georgia  Harris  46.718000
3          Georgia   Trump  47.240860
4         Michigan  Harris  47.841244
5         Michigan   Trump  45.413173
6           Nevada  Harris  47.307851
7           Nevada   Trump  46.263019
8   North Carolina  Harris  47.343665
9   North Carolina   Trump  47.197131
10    Pennsylvania  Harris  47.773305
11    Pennsylvania   Trump  46.239596
12       Wisconsin  Harris  48.548421
13       Wisconsin   Trump  45.642276


In [10]:
# Pivot table for margin
pivot_table = polling_averages.pivot(index="state", columns="answer", values="pct").reset_index()
pivot_table["margin"] = pivot_table["Harris"] - pivot_table["Trump"]

pivot_table.to_csv("data/swing_polling_pivot_data.csv", index=False)
print(pivot_table)

answer           state     Harris      Trump    margin
0              Arizona  46.646402  46.792305 -0.145903
1              Georgia  46.718000  47.240860 -0.522860
2             Michigan  47.841244  45.413173  2.428071
3               Nevada  47.307851  46.263019  1.044832
4       North Carolina  47.343665  47.197131  0.146533
5         Pennsylvania  47.773305  46.239596  1.533709
6            Wisconsin  48.548421  45.642276  2.906145


In [8]:
# Merge data from polling averages and pivot table
clean_data_with_margins = clean_data.merge(
    polling_averages,
    on=["state", "answer"],
    suffixes=("", "_average")
)
clean_data_with_margins = clean_data_with_margins.merge(
    pivot_table,
    on="state",
    how="left"
)
clean_data_with_margins = clean_data_with_margins[["state", "answer", "pct", "pct_average", "margin"]]
clean_data_with_margins.to_csv("data/polling_data_cleaned.csv", index=False)
clean_data_with_margins.head()

,state,answer,pct,pct_average,margin
0,Arizona,Harris,45.9,46.646402,-0.145903
1,Arizona,Trump,51.0,46.792305,-0.145903
2,Arizona,Harris,46.5,46.646402,-0.145903
3,Arizona,Trump,51.6,46.792305,-0.145903
4,Georgia,Harris,48.2,46.718000,-0.522860
